In [1]:
from rdflib_r2r import R2RStore, R2RMapping
from sqlalchemy import create_engine
from pathlib import Path
import rdflib
from rdflib.compare import to_isomorphic, graph_diff

echo = False


def create_database(db, path, *sql_fnames):
    for sql_fname in sql_fnames:
        sql_script = (path / sql_fname).open().read()
        conn = db.raw_connection()
        try:
            if hasattr(conn.connection, "c"):
                cursor = conn.connection.c
            else:
                cursor = conn.cursor()
            cursor.execute(sql_script, (), None)
        except Exception as e:
            try:
                cursor = conn.cursor()
                cursor.executescript(sql_script)
            except Exception as e:
                raise Exception(f"Problem with {path}: {e}")
            finally:
                conn.close()
        finally:
            conn.close()

# db = create_engine(
#     "duckdb:///:memory:",
#     echo=echo,
#     # future=True,
#     connect_args={"read_only": False},
# )
# db.dialect.server_version_info = (0,)

import sqlite3
db = create_engine(
    "sqlite:///:memory:",
    echo=echo,
    future=True,
    connect_args={
        "detect_types": sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
        'timeout': 30 * 1000
    },
    native_datetime=False, # breaks rdb2rdf test D016
    # native_datetime=True, # breaks BSBM test 3
)

path = Path('tests/rdb2rdf-ts/D014-3tables1primarykey1foreignkey')
sql_fname = 'create.sql'

sql_script = (path / sql_fname).open().read()
create_database(db, path, sql_fname)


# mapfile = path / 'r2rmla.ttl'
# fmt = rdflib.util.guess_format(str(mapfile))
# mapping = R2RMapping(rdflib.Graph().parse(str(mapfile), format=fmt))

mapping = R2RMapping.from_db(db)

g_made = rdflib.Graph(R2RStore(db=db, mapping=mapping))

print(g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}"))

@base <http://example.com/base/> .
@prefix ns1: <http://www.w3.org/ns/r2rml#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a ns1:TriplesMap ;
    ns1:logicalTable [ ns1:tableName "\"EMP\"" ] ;
    ns1:predicateObjectMap [ ns1:objectMap [ ns1:column "\"etype\"" ] ;
            ns1:predicate <EMP#etype> ],
        [ ns1:objectMap [ ns1:column "\"empno\"" ;
                    ns1:datatype xsd:integer ] ;
            ns1:predicate <EMP#empno> ],
        [ ns1:objectMap [ ns1:joinCondition [ ns1:child "\"deptno\"" ;
                            ns1:parent "\"deptno\"" ] ;
                    ns1:parentTriplesMap [ a ns1:TriplesMap ;
                            ns1:logicalTable [ ns1:tableName "\"DEPT\"" ] ;
                            ns1:predicateObjectMap [ ns1:objectMap [ ns1:column "\"deptno\"" ;
                                            ns1:datatype xsd:integer ] ;
                                    ns1:predicate <DEPT#deptno> ],
                                [ ns1:obj

SELECT anon_1.s AS s,
       anon_1.p AS p,
       anon_1.o AS o
FROM
  (SELECT CAST('_:DEPT#' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,
          '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' AS p,
          '<http://example.com/base/DEPT>' AS o,
          NULL AS g
   FROM "DEPT"
   UNION ALL SELECT CAST('_:DEPT#' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,
                    '<http://example.com/base/DEPT#dname>' AS p,
                    "DEPT".dname AS o,
                    NULL AS g
   FROM "DEPT"
   UNION ALL SELECT CAST('_:DEPT#' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,
                    '<http://example.com/base/DEPT#deptno>' AS p,
                    CAST('"' AS VARCHAR) || CAST(CAST("DEPT".deptno AS VARCHAR) AS VARCHAR) || CAST('"^^<http://www.w3.org/2001/XMLSchema#integer>' AS VARCHAR) AS o,
                    NULL AS g
   FROM "DEPT"
   UNION ALL SELECT CAST('_:DEPT#' AS VARCHAR) || CAS

In [2]:
with db.connect() as conn:
    for r in conn.execute(g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}")):
        print(r)

ObjectNotExecutableError: Not an executable object: 'SELECT anon_1.s AS s,\n       anon_1.p AS p,\n       anon_1.o AS o\nFROM\n  (SELECT CAST(\'_:DEPT#\' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,\n          \'<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>\' AS p,\n          \'<http://example.com/base/DEPT>\' AS o,\n          NULL AS g\n   FROM "DEPT"\n   UNION ALL SELECT CAST(\'_:DEPT#\' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,\n                    \'<http://example.com/base/DEPT#dname>\' AS p,\n                    "DEPT".dname AS o,\n                    NULL AS g\n   FROM "DEPT"\n   UNION ALL SELECT CAST(\'_:DEPT#\' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,\n                    \'<http://example.com/base/DEPT#deptno>\' AS p,\n                    CAST(\'"\' AS VARCHAR) || CAST(CAST("DEPT".deptno AS VARCHAR) AS VARCHAR) || CAST(\'"^^<http://www.w3.org/2001/XMLSchema#integer>\' AS VARCHAR) AS o,\n          

In [3]:
for t in g_made:
    print(t)

/Users/benno/Documents/projects/duckdb-sparql/r2r/rdflib_r2r/r2r_store.py:620: SAWarning: SELECT statement has a cartesian product between FROM element(s) "DEPT" and FROM element "EMP".  Apply join condition(s) between each element to resolve.
  rows = list(conn.execute(query))


(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/DEPT'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#dname'), rdflib.term.Literal('APPSERVER'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#deptno'), rdflib.term.Literal('10', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#loc'), rdflib.term.Literal('NEW YORK'))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/EMP'))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://example.com/base/EMP#etype'), rdflib.term.Literal('PART_TIME'))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URI

In [4]:
print(to_isomorphic(g_made).serialize())

/Users/benno/Documents/projects/duckdb-sparql/r2r/rdflib_r2r/r2r_store.py:620: SAWarning: SELECT statement has a cartesian product between FROM element(s) "EMP" and FROM element "DEPT".  Apply join condition(s) between each element to resolve.
  rows = list(conn.execute(query))


@prefix ns1: <http://example.com/base/EMP#> .
@prefix ns2: <http://example.com/base/LIKES#> .
@prefix ns3: <http://example.com/base/DEPT#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/base/EMP/empno=7369> a <http://example.com/base/EMP> ;
    ns1:deptno 10 ;
    ns1:empno 7369 ;
    ns1:ename "SMITH" ;
    ns1:etype "PART_TIME" ;
    ns1:job "CLERK" ;
    ns1:ref-deptno [ a <http://example.com/base/DEPT> ;
            ns3:deptno 10 ;
            ns3:dname "APPSERVER" ;
            ns3:loc "NEW YORK" ] .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Playing" ;
    ns2:likedObj "Soccer" .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Watching" ;
    ns2:likedObj "Basketball" .




In [5]:
outfile = path / 'directGraph.ttl'
# outfile = path / 'mappeda.nq'

fmt = rdflib.util.guess_format(str(outfile))
print(rdflib.Graph().load(str(outfile), format=fmt).serialize())

for t in rdflib.Graph().load(str(outfile), format=fmt):
    print(t)

@prefix ns1: <http://example.com/base/EMP#> .
@prefix ns2: <http://example.com/base/DEPT#> .
@prefix ns3: <http://example.com/base/LIKES#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/base/EMP/empno=7369> a <http://example.com/base/EMP> ;
    ns1:deptno 10 ;
    ns1:empno 7369 ;
    ns1:ename "SMITH" ;
    ns1:etype "PART_TIME" ;
    ns1:job "CLERK" ;
    ns1:ref-deptno [ a <http://example.com/base/DEPT> ;
            ns2:deptno 10 ;
            ns2:dname "APPSERVER" ;
            ns2:loc "NEW YORK" ] .

[] a <http://example.com/base/LIKES> ;
    ns3:id 7369 ;
    ns3:likeType "Playing" ;
    ns3:likedObj "Soccer" .

[] a <http://example.com/base/LIKES> ;
    ns3:id 7369 ;
    ns3:likeType "Watching" ;
    ns3:likedObj "Basketball" .


(rdflib.term.BNode('neab530fefb7e4ae4a0c8890dbc52ac59b1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/DEPT'))
(rdflib.term.URIRef('http://example.com/bas